In [2]:
import os
import csv
import random
import statistics
from collections import defaultdict
random.seed(0)


# Function to read data from a CSV file and organize it into a dictionary of lists
def read_csv_file(filename):
    data = defaultdict(list)  # Using defaultdict to automatically create lists for each time
    with open(filename, 'r') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            time = int(row[0])  # Extracting the time value from the first column
            value = float(row[1])  # Extracting the value from the second column
            data[time].append(value)  # Storing the value in the appropriate time slot
    return data

# Function to calculate bootstrap estimations for each time
def calculate_bootstrap_estimations(data, num_samples):
    bootstrap_estimations = defaultdict(list)  # Dictionary to store bootstrap estimations
    mean_values = {}  # Dictionary to store mean values

    for time, values in data.items():
        # Calculating the mean value for the original data
        mean_values[time] = statistics.mean(values)
        
        # Performing bootstrap resampling to estimate the distribution of means
        for _ in range(num_samples):
            sample = random.choices(values, k=len(values))  # Bootstrap resampling
            average = statistics.mean(sample)  # Calculating the mean of the resampled data
            bootstrap_estimations[time].append(average)  # Storing the estimation in the appropriate time slot
    
    return mean_values, bootstrap_estimations

# Function to generate a new filename with an incremented index
def generate_filename(base_filename, index):
    filename, extension = os.path.splitext(base_filename)
    new_filename = f"{filename}_{index}{extension}"
    return new_filename

# Function to write bootstrap estimation results to a CSV file
def write_bootstrap_results(filename, bootstrap_estimations):
    with open(filename, 'w', newline='') as file:
        csv_writer = csv.writer(file)
        # Writing header row with column names
        csv_writer.writerow(['Time'] + [f'Estimation_{i}' for i in range(len(bootstrap_estimations[0]))])
        # Writing data rows
        for time, estimations in bootstrap_estimations.items():
            csv_writer.writerow([time] + estimations)

if __name__ == "__main__":
    input_filename = input("Enter the input CSV filename: ")
    base_output_filename_bootstrap = "bootstrap_results.csv"
    base_output_filename_mean = "mean_results.csv"
    num_samples = int(input("Enter the number of bootstrap samples: "))
    
    data = read_csv_file(input_filename)
    
    # Finding the next available filename for bootstrap estimation results
    index_bootstrap = 1
    while os.path.exists(generate_filename(base_output_filename_bootstrap, index_bootstrap)):
        index_bootstrap += 1
    output_filename_bootstrap = generate_filename(base_output_filename_bootstrap, index_bootstrap)
    
    # Finding the next available filename for mean results
    index_mean = 1
    while os.path.exists(generate_filename(base_output_filename_mean, index_mean)):
        index_mean += 1
    output_filename_mean = generate_filename(base_output_filename_mean, index_mean)
    
    # Calculating mean values and bootstrap estimations
    mean_values, bootstrap_estimations = calculate_bootstrap_estimations(data, num_samples)
    
    # Writing bootstrap estimation results to a CSV file
    write_bootstrap_results(output_filename_bootstrap, bootstrap_estimations)

    # Writing mean values to a CSV file
    with open(output_filename_mean, 'w', newline='') as file:
        csv_writer = csv.writer(file)
        csv_writer.writerow(['Time', 'Mean Value'])
        for time, mean in mean_values.items():
            csv_writer.writerow([time, mean])

    print("Bootstrap estimations saved to", output_filename_bootstrap)
    print("Mean values saved to", output_filename_mean)


Enter the input CSV filename: 1.csv
Enter the number of bootstrap samples: 50
Bootstrap estimations saved to bootstrap_results_4.csv
Mean values saved to mean_results_4.csv
